In [ ]:
import requests
import json
import time
import re
import csv

In [ ]:
#Must have OAuth Token from Github
with open('myOAuthToken.txt', 'r') as f:
    oauthtoken = f.read()
authorization = requests.auth.HTTPBasicAuth(oauthtoken, '')

In [ ]:
def itsgithubcom(url):
    return re.search(r'github.com/', url)

def makerepoquery(url):
    base = 'https://api.github.com/repos/'
    spliturl = url.split('/')
    repo = spliturl[3] + '/' + spliturl[4]
    apiquery = base + repo
    return apiquery

In [ ]:
#define outputs
listtowrite = []
repocolumnstoadd = ['name', 'created_at', 'updated_at', 'size',
                    'stargazers_count', 'watchers_count',
                    'language', 'forks_count', 'has_wiki',
                    'has_pages', 'subscribers_count']
ownercolumnstoadd = ['type', 'name', 'company', 'blog',
                     'location', 'email', 'bio', 'public_repos',
                     'public_gists', 'followers', 'following',
                     'created_at', 'updated_at']

In [ ]:
#get the results
with open('ZenodoSoftwareGithubData4-7-2015.csv', 'r') as f:
    mycsv = csv.reader(f)
    for row in mycsv:
        url = row[9]
        if itsgithubcom(url):
            apiquery = makerepoquery(url)
            results = requests.get(apiquery, auth=authorization)
            jsonresults = json.loads(results.text)
            if 'owner' in jsonresults:
                ownerquery = jsonresults['owner']['url']
                ownerresults = requests.get(ownerquery, auth=authorization)
                jsonownerresults = json.loads(ownerresults.text)
                row.append(apiquery)
                for column in repocolumnstoadd:
                    if column in jsonresults:
                        row.append(jsonresults[column])
                    else:
                        row.append('')
                for column2 in ownercolumnstoadd:
                    if column2 in jsonownerresults:
                        row.append(jsonownerresults[column2])
                    else:
                        row.append('')
                print(row)
            else:
                row.append('manual lookup needed')
                print('manual lookup needed')
        else:
            print('not repo')
        listtowrite.append(row)
        time.sleep(2)
f.closed

In [ ]:
#make the csv
with open('ZenodoGithubOutput4-15-2015.csv', 'wb') as f:
    for i in listtowrite:
        for j in i:
            try:
                f.write('\"' + str(j).replace('"', "'") + '\"' + ',')
            except:
                f.write('\"' + j.encode('UTF-8').replace('"', "'") + '\"' + ',')
        f.write('\r\n')